In [1]:
import pandas as pd
import numpy 
from tqdm import tqdm 
from scripts.utils import load_model

In [2]:
tokenizer, _ = load_model('llama7b')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# tokenizer.decode(204)

In [3]:
metric_data_path = '/scratch/ramprasad.sa/probing_summarization_factuality/metric_scores/scores_llama7b_ACIBENCH.csv'
df = pd.read_csv(metric_data_path)
df = df[df['target_token_id'] != 204]
df.head()

,Unnamed: 0,target_token_id,factual_label,target_token_str,P_y|x,P_y,P_ytopic,score_pmi,score_pmi_topic,distance_pmi,...,score_poel,score_poel_topic,score_harim,score_harim_topic,score_harim_plus,score_harim_plus_topic,score_coco_token,score_coco_span,score_coco_sent,document_id
0,0,450,0,The,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-1.148615,-1.148615,2.0,...,0.435563,0.435563,0.845033,0.845033,-8.371784,-8.371784,1.000000,1.000000,1.000000,Emily Edwards#ACIBENCH-D2N123#1702274615:llama...
1,1,16500,0,patient,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-3.615121,-3.615121,2.0,...,-2.030942,-2.030942,-0.000000,-0.000000,0.009950,0.009950,0.891924,0.202812,1.000000,Emily Edwards#ACIBENCH-D2N123#1702274615:llama...
2,2,29892,0,",","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-3.615121,-3.615121,2.0,...,-2.030942,-2.030942,-0.000000,-0.000000,0.009950,0.009950,1.000000,1.000000,1.000000,Emily Edwards#ACIBENCH-D2N123#1702274615:llama...
3,3,9656,0,Ty,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-3.615121,-3.615121,2.0,...,-2.030942,-2.030942,-0.000000,-0.000000,0.009950,0.009950,0.000000,0.000000,0.000000,Emily Edwards#ACIBENCH-D2N123#1702274615:llama...
4,4,1358,0,ler,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.000000,1.000000,0.0,...,0.996577,0.996577,-0.010000,-0.010000,0.079950,0.079950,1.000000,1.000000,0.498891,Emily Edwards#ACIBENCH-D2N123#1702274615:llama...


In [4]:
len(df)

6924

In [16]:
# list(df['target_token_str'].values)

In [5]:
df = df[~df['target_token_str'].isnull()]
unique_documents = list(set(df['document_id'].values))
unique_documents[:1], len(unique_documents)

(['Cynthia Lamanda#ACIBENCH-D2N107#1701505474:llama7b-ul2'], 51)

In [6]:
len(unique_documents)

51

In [7]:
df_document_word = {
    'word': [],
    'P_y|x': [],
    'P_y': [],
    'P_ytopic': [],
    'score_pmi': [],
    'score_pmi_topic': [],
    'distance_pmi': [],
    'distance_pmi_topic': [],
    'score_poel': [],
    'score_poel_topic': [],
    'score_harim': [],
    'score_harim_topic': [],
    'score_harim_plus': [],
    'score_harim_plus_topic': [],
    'score_coco_token': [],
    'score_coco_span': [],
    'score_coco_sent': [],
    'factual_label': []
}



for document_id in tqdm(unique_documents):
    df_document = df[df['document_id'] == document_id]
    words_scores = {}
    words = []
    scores = {}
    row_counter = 0
    for idx, row in df_document.iterrows():
            target_token_str = row['target_token_str']
            # if target_token_str.strip():
            reset = False
            word = ''.join(words)
            
            if row_counter == 0:
                target_token_str = target_token_str.strip()
                
            if (' ' in target_token_str) or (idx == len(df_document) - 1):
                reset = True

            if reset:
                if word.strip():
                    df_document_word['word']+= [word]
                    for score_key, score_list in scores.items():
                        df_document_word[score_key].append(score_list)
                words = []
                scores = {}

            words += [target_token_str]
            for column in row.keys():
                target_token_score = None
                if column in ['P_y|x', 'P_y', 'P_ytopic']:
                        prob_score = numpy.array(eval(row[column]))
                        target_token_score = prob_score[row['target_token_id']]
                elif 'distance' in column or 'score' in column or 'factual_label' in column:
                        # print(row[column])
                        target_token_score = row[column]
                if target_token_score != None:
                    if column not in scores: 
                            scores[column] = []
                    scores[column] += [target_token_score]
            row_counter += 1

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 51/51 [13:37<00:00, 16.03s/it]


In [8]:
{k : len(v) for k , v in df_document_word.items()}

{'word': 4672,
 'P_y|x': 4672,
 'P_y': 4672,
 'P_ytopic': 4672,
 'score_pmi': 4672,
 'score_pmi_topic': 4672,
 'distance_pmi': 4672,
 'distance_pmi_topic': 4672,
 'score_poel': 4672,
 'score_poel_topic': 4672,
 'score_harim': 4672,
 'score_harim_topic': 4672,
 'score_harim_plus': 4672,
 'score_harim_plus_topic': 4672,
 'score_coco_token': 4672,
 'score_coco_span': 4672,
 'score_coco_sent': 4672,
 'factual_label': 4672}

In [9]:
df_document_word = pd.DataFrame(df_document_word)
{k : len(v) for k , v in df_document_word.items()}
df_document_word.head()

,word,P_y|x,P_y,P_ytopic,score_pmi,score_pmi_topic,distance_pmi,distance_pmi_topic,score_poel,score_poel_topic,score_harim,score_harim_topic,score_harim_plus,score_harim_plus_topic,score_coco_token,score_coco_span,score_coco_sent,factual_label
0,The,[0.11041267961263657],[0.0],[0.0],[-1.4883397467330268],[-1.4883397467330268],[2.00000001816079],[0.2208254635334015],[0.0958387972468761],[0.0958387972468761],[0.7824697273900959],[0.7824697273900959],[-7.594118530985735],[-7.594118530985735],[1.0],[1.0],[1.0],[0]
1,"patient,","[1.0, 0.11917408555746078]","[0.0, 0.0]","[0.0, 0.0]","[-3.615120516841259, -1.5585759020557193]","[-3.615120516841259, -1.5585759020557193]","[2.000000006519258, 2.000000014901161]","[1.999999925494194, 1.999999962747097]","[-2.030941972861356, 0.0256026419241837]","[-2.030941972861356, 0.0256026419241837]","[-0.0, 0.7670460324091101]","[-0.0, 0.7670460324091101]","[0.009950330853168, -7.4159165107961424]","[0.009950330853168, -7.4159165107961424]","[1.0, 1.0]","[1.0, 1.0]","[1.0, 1.0]","[0, 0]"
2,"Bryan,","[1.0, 1.0, 1.0]","[0.0, 0.6291239857673645, 1.0]","[0.0, 0.4255390465259552, 1.0]","[-3.615120516841259, 0.5423928566555052, 1.0]","[-3.615120516841259, 0.1588788415103458, 1.0]","[2.0000000298023224, 0.7417520955204964, 0.0]","[2.000000026077032, 1.148921936750412, 0.0]","[-2.030941972861356, 0.6963868001525215, 0.996...","[-2.030941972861356, 0.444801606217297, 0.9965...","[-0.0, -0.0062912398576736, -0.01]","[-0.0, -0.0042553904652595, -0.01]","[0.009950330853168, 0.0539890098568836, 0.0799...","[0.009950330853168, 0.0397380641099849, 0.0799...","[0.0, 0.8404759168624878, 1.0]","[0.0, 0.8769726157188416, 1.0]","[0.0, 0.8494499325752258, 1.0]","[0, 0, 0]"
3,is,[0.0],[0.18700164556503296],[0.15148496627807617],[3.98062698883105],[3.781826957274255],[1.9999999850988388],[1.5275070518255234],[4.539469848653072],[4.409057027951814],[1.1751316291093823],[1.1399701166152951],[-12.831091589753768],[-12.58496100229516],[1.42728653550148],[1.0],[1.021860159933567],[0]
4,experiencing,"[1.0, 1.0]","[0.0, 1.0]","[0.1514168381690979, 1.0]","[-3.615120516841259, 1.0]","[-0.8337155337340785, 1.0]","[1.999999951571226, 0.0]","[1.6971663311123848, 0.0]","[-2.030941972861356, 0.9965770861865102]","[-0.2063403039430453, 0.9965770861865102]","[-0.0, -0.01]","[-0.0015141683816909, -0.01]","[0.009950330853168, 0.0799503308531681]","[0.0205495095250049, 0.0799503308531681]","[1.0, 1.0]","[1.0, 1.0]","[1.0, 1.0]","[0, 0]"


In [10]:
import numpy as np
from sklearn import metrics

def get_scores(df, metric_key):
    scoring_criteria = {'max': [],
                       'min': [],
                        'mean': [],
                        'first': []
                       }
    all_labels = []
    for idx, row in df.iterrows():
        label = row['factual_label']
        if len(set(label))== 1:
            metric_value = row[metric_key]
            scoring_criteria['max'] +=  [max(metric_value)]
            scoring_criteria['min'] += [min(metric_value)]
            scoring_criteria['mean'] += [np.mean(metric_value)]
            scoring_criteria['first'] += [metric_value[0]]
            
            # print(row)
            all_labels += list(set(label))
        # else:
        #     print(row)
    
    results = {}
    
    for criteria, pred_scores in scoring_criteria.items():
        criteria_score = metrics.roc_auc_score(all_labels, pred_scores)
        results[criteria] = criteria_score
#     for criteria, pred_scores in scoring_criteria.items():
#         fpr, tpr, thresholds = metrics.roc_curve(all_labels, pred_scores)
#         assert(len(all_labels) == len(pred_scores))
#         fpr_values = fpr
#         tpr_values = tpr
#         fnr_values = 1 - tpr
#         tnr_values = 1 - fpr_values
#         balanced_accuracy = (tpr_values + tnr_values) / 2
#         for i in range(len(thresholds)):
#             print("Threshold: {:.2f}, FPR: {:.2f}, FNR: {:.2f}, BA: {:.2f}".format(thresholds[i], fpr[i], tpr[i], balanced_accuracy[i]))
    print(results)
    
            

In [11]:
get_scores(df_document_word, 'P_y|x')

{'max': 0.46750087782564964, 'min': 0.4586342456393418, 'mean': 0.4586905925560385, 'first': 0.4480522946836981}


In [12]:
get_scores(df_document_word, 'score_pmi')

{'max': 0.5484891908820013, 'min': 0.5674051473798092, 'mean': 0.5651512707119404, 'first': 0.567078928388407}


In [13]:
get_scores(df_document_word, 'score_poel')

{'max': 0.550280429707518, 'min': 0.5681429954258167, 'mean': 0.5651311044470173, 'first': 0.571467463510923}


In [14]:
get_scores(df_document_word, 'score_harim')

{'max': 0.5032574449105093, 'min': 0.5032859149315771, 'mean': 0.5048043160551938, 'first': 0.5122142321635318}


In [15]:
get_scores(df_document_word, 'score_coco_token')

{'max': 0.5173572228443448, 'min': 0.560527857915615, 'mean': 0.5578184609106611, 'first': 0.5538528242260898}


In [16]:
get_scores(df_document_word, 'score_coco_span')

{'max': 0.511348269022719, 'min': 0.5502626359443505, 'mean': 0.5463658018106934, 'first': 0.5463580911799875}


In [17]:
get_scores(df_document_word, 'score_coco_sent')

{'max': 0.5102017575493005, 'min': 0.557390817469205, 'mean': 0.547871747300093, 'first': 0.5464583293791637}
